# Dataset

> A python list like object that contains your evaluation data.

In [1]:
# | default_exp dataset

In [2]:
# | export
from abc import ABC, abstractmethod
import os
import typing as t
import csv
import uuid

from fastcore.utils import patch
import pandas as pd

from ragas_experimental.model.pydantic_model import ExtendedPydanticBaseModel as BaseModel
from ragas_experimental.utils import create_nano_id, async_to_sync, get_test_directory
from ragas_experimental.backends.ragas_api_client import RagasApiClient
from ragas_experimental.typing import SUPPORTED_BACKENDS
import ragas_experimental.typing as rt
from ragas_experimental.metric import MetricResult

/Users/jjmachan/workspace/eglabs/ragas/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#| export
BaseModelType = t.TypeVar("BaseModelType", bound=BaseModel)

class DatasetBackend(ABC):
    """Abstract base class for dataset backends.
    
    All dataset storage backends must implement these methods.
    """
    
    @abstractmethod
    def initialize(self, dataset): 
        """Initialize the backend with dataset information"""
        pass
    
    @abstractmethod
    def get_column_mapping(self, model): 
        """Get mapping between model fields and backend columns"""
        pass
    
    @abstractmethod
    def load_entries(self, model_class): 
        """Load all entries from storage"""
        pass
    
    @abstractmethod
    def append_entry(self, entry): 
        """Add a new entry to storage and return its ID"""
        pass
    
    @abstractmethod
    def update_entry(self, entry): 
        """Update an existing entry in storage"""
        pass
    
    @abstractmethod
    def delete_entry(self, entry_id): 
        """Delete an entry from storage"""
        pass
    
    @abstractmethod
    def get_entry_by_field(self, field_name: str, field_value: t.Any, model_class):
        """Get an entry by field value"""
        pass



In [4]:
#| export
class RagasAppBackend(DatasetBackend):
    """Backend for storing datasets using the Ragas API."""
    
    def __init__(self, ragas_api_client, project_id, dataset_id):
        """Initialize the RagasAppBackend.
        
        Args:
            ragas_api_client: The RagasApiClient instance
            project_id: The ID of the project
            dataset_id: The ID of the dataset
        """
        self.ragas_api_client = ragas_api_client
        self.project_id = project_id
        self.dataset_id = dataset_id
        self.dataset = None

    def __str__(self):
        return f"RagasAppBackend(project_id={self.project_id}, dataset_id={self.dataset_id})"

    def __repr__(self):
        return self.__str__()
        
    def initialize(self, dataset):
        """Initialize the backend with the dataset instance."""
        self.dataset = dataset
        
    def get_column_mapping(self, model):
        """Get mapping between model fields and backend columns."""
        sync_func = async_to_sync(self.ragas_api_client.list_dataset_columns)
        columns = sync_func(project_id=self.project_id, dataset_id=self.dataset_id)
        column_id_map = {column["name"]: column["id"] for column in columns["items"]}
        
        # Update the model's column mapping with the values from the API
        column_mapping = {}
        for field_name in model.__annotations__:
            if field_name in column_id_map:
                column_mapping[field_name] = column_id_map[field_name]
        
        return column_mapping
    
    def load_entries(self, model_class):
        """Load all entries from the API."""
        # Get all rows
        sync_func = async_to_sync(self.ragas_api_client.list_dataset_rows)
        response = sync_func(
            project_id=self.project_id,
            dataset_id=self.dataset_id
        )
        
        # Get column mapping (ID -> name)
        column_map = {v: k for k, v in model_class.__column_mapping__.items()}
        
        # Process rows
        entries = []
        for row in response.get("items", []):
            model_data = {}
            row_id = row.get("id")
            
            # Convert from API data format to model fields
            for col_id, value in row.get("data", {}).items():
                if col_id in column_map:
                    field_name = column_map[col_id]
                    model_data[field_name] = value
            
            # Create model instance
            entry = model_class(**model_data)
            
            # Store row ID for future operations
            entry._row_id = row_id
            
            entries.append(entry)
            
        return entries
    
    def append_entry(self, entry):
        """Add a new entry to the API and return its ID."""
        import ragas_experimental.typing as rt
        
        # Get column mapping
        column_id_map = entry.__class__.__column_mapping__
        
        # Create row data
        row_dict_converted = rt.ModelConverter.instance_to_row(entry)
        row_id = create_nano_id()
        row_data = {}
        
        for column in row_dict_converted["data"]:
            if column["column_id"] in column_id_map:
                row_data[column_id_map[column["column_id"]]] = column["data"]
        
        # Create row in API
        sync_func = async_to_sync(self.ragas_api_client.create_dataset_row)
        response = sync_func(
            project_id=self.project_id,
            dataset_id=self.dataset_id,
            id=row_id,
            data=row_data,
        )
        
        # Return the row ID
        return response["id"]
    
    def update_entry(self, entry):
        """Update an existing entry in the API."""
        import ragas_experimental.typing as rt
        
        # Get the row ID
        row_id = None
        if hasattr(entry, "_row_id") and entry._row_id:
            row_id = entry._row_id
        else:
            raise ValueError("Cannot update: entry has no row ID")
        
        # Get column mapping and prepare data
        column_id_map = entry.__class__.__column_mapping__
        row_dict = rt.ModelConverter.instance_to_row(entry)["data"]
        row_data = {}
        
        for column in row_dict:
            if column["column_id"] in column_id_map:
                row_data[column_id_map[column["column_id"]]] = column["data"]
        
        # Update in API
        sync_func = async_to_sync(self.ragas_api_client.update_dataset_row)
        response = sync_func(
            project_id=self.project_id,
            dataset_id=self.dataset_id,
            row_id=row_id,
            data=row_data,
        )
        
        return response
    
    def delete_entry(self, entry_id):
        """Delete an entry from the API."""
        # Delete the row
        sync_func = async_to_sync(self.ragas_api_client.delete_dataset_row)
        response = sync_func(
            project_id=self.project_id,
            dataset_id=self.dataset_id,
            row_id=entry_id
        )
        
        return response
    
    def get_entry_by_field(self, field_name, field_value, model_class):
        """Get an entry by field value."""
        # We don't have direct filtering in the API, so load all and filter
        entries = self.load_entries(model_class)
        
        # Search for matching entry
        for entry in entries:
            if hasattr(entry, field_name) and getattr(entry, field_name) == field_value:
                return entry
                
        return None


In [5]:
#| export
class LocalBackend(DatasetBackend):
    def __init__(self, local_root_dir, project_id, dataset_id, dataset_name, type: t.Literal["datasets", "experiments"]):
        """Initialize the LocalBackend.
        
        Args:
            local_root_dir: The root directory for all projects
            project_id: The ID of the project
            dataset_id: The ID of the dataset
            dataset_name: The name of the dataset
        """
        self.local_root_dir = local_root_dir
        self.project_id = project_id
        self.dataset_id = dataset_id
        self.dataset_name = dataset_name
        self.dataset = None
        self.type = type

    def __str__(self):
        return f"LocalBackend(local_root_dir={self.local_root_dir}, project_id={self.project_id}, dataset_id={self.dataset_id}, dataset_name={self.dataset_name})"

    def __repr__(self):
        return self.__str__()
        
    def initialize(self, dataset):
        """Initialize the backend with the dataset instance."""
        self.dataset = dataset
        
        # Ensure CSV file exists
        self._ensure_csv_exists()
        
    def _ensure_csv_exists(self):
        """Create the CSV file if it doesn't exist."""
        csv_path = self._get_csv_path()
        
        # Create directories if needed
        os.makedirs(os.path.dirname(csv_path), exist_ok=True)
        
        # Create file with headers if it doesn't exist
        if not os.path.exists(csv_path):
            # Include _row_id in the headers
            field_names = ["_row_id"] + list(self.dataset.model.__annotations__.keys())
            
            with open(csv_path, 'w', newline='') as f:
                writer = csv.writer(f)
                writer.writerow(field_names)
    
    def _get_csv_path(self):
        """Get the path to the CSV file."""
        return os.path.join(
            self.local_root_dir, 
            self.project_id, 
            self.type,
            f"{self.dataset_name}.csv"
        )
        
    def get_column_mapping(self, model) -> t.Dict:
        """Get mapping between model fields and CSV columns.
        
        For CSV, column names directly match field names.
        """
        # Simple dictionary comprehension
        return model.model_fields
    
    def load_entries(self, model_class):
        """Load all entries from the CSV file."""
        csv_path = self._get_csv_path()
        
        if not os.path.exists(csv_path):
            return []
            
        entries = []
        
        with open(csv_path, 'r', newline='') as f:
            reader = csv.DictReader(f)
            
            for row in reader:
                try:
                    # Extract row_id and remove from model data
                    row_id = row.get("_row_id", str(uuid.uuid4()))
                    
                    # Create a copy without _row_id for model instantiation
                    model_data = {k: v for k, v in row.items() if k != "_row_id"}
                    
                    # Convert types as needed
                    typed_row = {}
                    for field, value in model_data.items():
                        if field in model_class.model_fields:
                            field_type = model_class.model_fields[field].annotation
                            
                            # Handle basic type conversions
                            if field_type == int:
                                typed_row[field] = int(value) if value else 0
                            elif field_type == float:
                                typed_row[field] = float(value) if value else 0.0
                            elif field_type == bool:
                                typed_row[field] = value.lower() in ('true', 't', 'yes', 'y', '1')
                            else:
                                typed_row[field] = value
                    
                    # Create model instance
                    entry = model_class(**typed_row)
                    
                    # Set the row ID from CSV (or use UUID if not available)
                    entry._row_id = row_id
                    
                    entries.append(entry)
                except Exception as e:
                    print(f"Error loading row from CSV: {e}")
        
        return entries
    
    def append_entry(self, entry):
        """Add a new entry to the CSV file and return a generated ID."""
        csv_path = self._get_csv_path()
        
        # Read existing rows to avoid overwriting
        existing_rows = []
        if os.path.exists(csv_path) and os.path.getsize(csv_path) > 0:
            with open(csv_path, 'r', newline='') as f:
                reader = csv.DictReader(f)
                existing_rows = list(reader)
        
        # Generate a row ID if needed
        row_id = getattr(entry, "_row_id", None) or str(uuid.uuid4())
        
        # Get field names including row_id
        field_names = ["_row_id"] + list(entry.model_fields.keys())
        
        # Convert entry to dict
        entry_dict = entry.model_dump()
        
        # Add row_id to the dict
        entry_dict["_row_id"] = row_id
        
        # Write all rows back with the new entry
        with open(csv_path, 'w', newline='') as f:
            writer = csv.DictWriter(f, fieldnames=field_names)
            writer.writeheader()
            
            # Write existing rows
            for row in existing_rows:
                writer.writerow(row)
                
            # Write new row
            writer.writerow(entry_dict)
        
        # Return the row ID
        return row_id
    
    def update_entry(self, entry):
        """Update an existing entry in the CSV file.
        
        Since CSV files don't support in-place updates, we need to
        rewrite the entire file.
        """
        # Create a copy of entries to modify
        entries_to_save = list(self.dataset._entries)  # Make a copy
        
        # Find the entry to update
        updated = False
        for i, e in enumerate(entries_to_save):
            if hasattr(e, "_row_id") and hasattr(entry, "_row_id") and e._row_id == entry._row_id:
                # Update the entry in our copy
                entries_to_save[i] = entry
                updated = True
                break
        
        # If entry wasn't found, just append it
        if not updated and entries_to_save:
            entries_to_save.append(entry)
        
        # Write all entries back to CSV
        self._write_entries_to_csv(entries_to_save)
        
        return True
    
    def delete_entry(self, entry_id):
        """Delete an entry from the CSV file.
        
        This method should NOT modify self.dataset._entries directly.
        Dataset.pop() handles that separately.
        """
        # Create a copy of entries to modify, excluding the one to delete
        entries_to_save = []
        for e in self.dataset._entries:
            if not (hasattr(e, "_row_id") and e._row_id == entry_id):
                entries_to_save.append(e)
        
        # Write all entries back to CSV
        self._write_entries_to_csv(entries_to_save)
        
        return True
    
    def _write_entries_to_csv(self, entries):
        """Write all entries to the CSV file."""
        csv_path = self._get_csv_path()
        
        if not entries:
            # If no entries, just create an empty CSV with headers
            field_names = ["_row_id"] + list(self.dataset.model.model_fields.keys())
            with open(csv_path, 'w', newline='') as f:
                writer = csv.DictWriter(f, fieldnames=field_names)
                writer.writeheader()
            return
            
        # Get field names including _row_id
        field_names = ["_row_id"] + list(entries[0].__class__.model_fields.keys())
        
        # Write all entries
        with open(csv_path, 'w', newline='') as f:
            writer = csv.DictWriter(f, fieldnames=field_names)
            writer.writeheader()
            
            for entry in entries:
                # Create a dict with model data + row_id
                entry_dict = entry.model_dump()
                entry_dict["_row_id"] = getattr(entry, "_row_id", str(uuid.uuid4()))
                
                writer.writerow(entry_dict)
    
    def get_entry_by_field(self, field_name, field_value, model_class):
        """Get an entry by field value."""
        entries = self.load_entries(model_class)
        
        for entry in entries:
            if hasattr(entry, field_name) and getattr(entry, field_name) == field_value:
                return entry
                
        return None

In [6]:
#| export
def create_dataset_backend(backend_type: SUPPORTED_BACKENDS, **kwargs):
    """Factory function to create the appropriate backend.
    
    Args:
        backend_type: The type of backend to create (ragas_app or local)
        **kwargs: Arguments specific to the backend
        
    Returns:
        DatasetBackend: An instance of the requested backend
    """
    backend_classes = {
        "ragas_app": RagasAppBackend,
        "local": LocalBackend,
    }
    
    if backend_type not in backend_classes:
        raise ValueError(f"Unsupported backend: {backend_type}")
        
    return backend_classes[backend_type](**kwargs)

In [7]:
temp_dir = get_test_directory()
backend = create_dataset_backend("local", local_root_dir=temp_dir, project_id="test_project", dataset_id="test_dataset", dataset_name="test_dataset", type="dataset")
print(backend)

LocalBackend(local_root_dir=/var/folders/2y/02fp70k56p75ldrkgtx7z10r0000gn/T/ragas_test_dPmBLc9qLgdj, project_id=test_project, dataset_id=test_dataset, dataset_name=test_dataset)


In [8]:
# | export
class Dataset(t.Generic[BaseModelType]):
    """A list-like interface for managing dataset entries with backend synchronization.
    
    This class behaves like a Python list while synchronizing operations with the
    chosen backend (Ragas API or local filesystem).
    """

    def __init__(
        self,
        name: str,
        model: t.Type[BaseModel],
        project_id: str,
        dataset_id: str,
        datatable_type: t.Literal["datasets", "experiments"],
        ragas_api_client: t.Optional[RagasApiClient] = None,
        backend: SUPPORTED_BACKENDS = "local",
        local_root_dir: t.Optional[str] = None,
    ):
        """Initialize a Dataset with the specified backend.
        
        Args:
            name: The name of the dataset
            model: The Pydantic model class for entries
            project_id: The ID of the parent project
            dataset_id: The ID of this dataset
            ragas_api_client: Required for ragas_app backend
            backend: The storage backend to use (ragas_app or local)
            local_root_dir: Required for local backend
        """
        # Store basic properties
        self.name = name
        self.model = model
        self.project_id = project_id
        self.dataset_id = dataset_id
        self.backend_type = backend
        self.datatable_type = datatable_type
        self._entries: t.List[BaseModelType] = []

        # Create the appropriate backend
        backend_params = {}
        if backend == "ragas_app":
            if ragas_api_client is None:
                raise ValueError("ragas_api_client is required for ragas_app backend")
            backend_params = {
                "ragas_api_client": ragas_api_client,
                "project_id": project_id,
                "dataset_id": dataset_id
            }
        elif backend == "local":
            if local_root_dir is None:
                raise ValueError("local_root_dir is required for local backend")
            backend_params = {
                "local_root_dir": local_root_dir,
                "project_id": project_id,
                "dataset_id": dataset_id,
                "dataset_name": name,
                "type": self.datatable_type
            }
            
        self._backend = create_dataset_backend(backend, **backend_params)
        
        # Initialize the backend with this dataset
        self._backend.initialize(self)
        
        # Initialize column mapping if it doesn't exist yet
        if not hasattr(self.model, "__column_mapping__"):
            self.model.__column_mapping__ = {}
            
        # Get column mappings from backend and update the model's mapping
        column_mapping = self._backend.get_column_mapping(model)
        
        # Update the model's column mapping
        for field_name, column_id in column_mapping.items():
            self.model.__column_mapping__[field_name] = column_id

    def __getitem__(
        self, key: t.Union[int, slice]
    ) -> t.Union[BaseModelType, "Dataset[BaseModelType]"]:
        """Get an entry by index or slice."""
        if isinstance(key, slice):
            # Create a new dataset with the sliced entries
            new_dataset = type(self)(
                name=self.name,
                model=self.model,
                project_id=self.project_id,
                dataset_id=self.dataset_id,
                backend=self.backend_type,
                datatable_type=self.datatable_type
            )
            # Copy the backend reference
            new_dataset._backend = self._backend
            # Set the entries to the sliced entries
            new_dataset._entries = self._entries[key]
            return new_dataset
        else:
            return self._entries[key]

    def __setitem__(self, index: int, entry: BaseModelType) -> None:
        """Update an entry at the given index and sync to backend."""
        if not isinstance(entry, self.model):
            raise TypeError(f"Entry must be an instance of {self.model.__name__}")

        # Get existing entry to get its ID
        existing = self._entries[index]
        if hasattr(existing, "_row_id") and existing._row_id:
            entry._row_id = existing._row_id
        
        # Update in backend
        self._backend.update_entry(entry)
        
        # Update local cache
        self._entries[index] = entry

    def __repr__(self) -> str:
        """String representation of the dataset."""
        return f"Dataset(name='{self.name}', model={self.model.__name__}, len={len(self)})"

    def __len__(self) -> int:
        """Get the number of entries in the dataset."""
        return len(self._entries)

    def __iter__(self) -> t.Iterator[BaseModelType]:
        """Iterate over the entries in the dataset."""
        return iter(self._entries)

In [9]:
# test model
class DatasetModel(BaseModel):
    id: int
    name: str
    description: str

class ExperimentModel(DatasetModel):
    tags: t.Literal["tag1", "tag2", "tag3"]
    result: MetricResult

In [10]:
dataset_instance = DatasetModel(
    id=0, 
    name="test", 
    description="test description", 
)
dataset_instance

DatasetModel(id=0, name='test', description='test description')

In [11]:
experiment_instance = ExperimentModel(
    **dataset_instance.model_dump(),
    tags="tag1",
    result=MetricResult(result=0.5, reason="test reason"),
)
experiment_instance

ExperimentModel(id=0, name='test', description='test description', tags='tag1', result=0.5)

In [12]:
from ragas_experimental import Project

In [13]:
tmp_root_dir = get_test_directory()

In [14]:
p = Project.create(name="test_project", backend="local", root_dir=tmp_root_dir)
dataset_with_dataset_model = p.create_dataset(name="dataset_with_dataset_model", model=DatasetModel)
dataset_with_experiment_model = p.create_dataset(name="dataset_with_experiment_model", model=ExperimentModel)

In [15]:
experiment_instance = ExperimentModel(
    **dataset_instance.model_dump(),
    tags="tag1",
    result=MetricResult(result=0.5, reason="test reason"),
)
experiment_instance

ExperimentModel(id=0, name='test', description='test description', tags='tag1', result=0.5)

In [16]:
# | export
@patch
def append(self: Dataset, entry: BaseModelType) -> None:
    """Add a new entry to the dataset and sync to backend.
    
    Args:
        entry: The entry to add to the dataset
    """
    if not isinstance(entry, self.model):
        raise TypeError(f"Entry must be an instance of {self.model.__name__}")
        
    # Add to backend and get ID
    row_id = self._backend.append_entry(entry)
    
    # Store the ID
    entry._row_id = row_id
    
    # Add to local cache
    self._entries.append(entry)

In [17]:
dataset_with_dataset_model.append(dataset_instance)
dataset_with_experiment_model.append(experiment_instance)
len(dataset_with_dataset_model), len(dataset_with_experiment_model)

(1, 1)

In [18]:
from fastcore.test import *

In [19]:
# | hide
test_eq(len(dataset_with_dataset_model), 1)
test_eq(len(dataset_with_experiment_model), 1)

In [20]:
# | export
@patch
def pop(self: Dataset, index: int = -1) -> BaseModelType:
    """Remove and return entry at index, sync deletion to backend.
    
    Args:
        index: The index of the entry to remove (default: -1, the last entry)
        
    Returns:
        The removed entry
    """
    # Get the entry
    entry = self._entries[index]
    
    # Get the row ID
    row_id = getattr(entry, "_row_id", None)
    if row_id is None:
        raise ValueError("Entry has no row ID. This likely means it was not added or synced to the dataset.")
    
    # Delete from backend
    self._backend.delete_entry(row_id)
    
    # Remove from local cache
    return self._entries.pop(index)

In [21]:
dataset_with_dataset_model.pop()
dataset_with_experiment_model.pop()
len(dataset_with_dataset_model), len(dataset_with_experiment_model)

(0, 0)

In [22]:
# | hide
test_eq(len(dataset_with_dataset_model), 0)
test_eq(len(dataset_with_experiment_model), 0)

In [23]:
# now add some more entries
for i in range(10):
    dataset_with_dataset_model.append(dataset_instance)
    dataset_with_experiment_model.append(experiment_instance)

test_eq(len(dataset_with_dataset_model), 10)
test_eq(len(dataset_with_experiment_model), 10)

In [24]:
# | export
@patch
def load(self: Dataset) -> None:
    """Load all entries from the backend."""
    # Get entries from backend
    self._entries = self._backend.load_entries(self.model)

In [25]:
dataset_with_dataset_model.load()
dataset_with_experiment_model.load()

In [26]:
# | export
@patch
def load_as_dicts(self: Dataset) -> t.List[t.Dict]:
    """Load all entries as dictionaries.
    
    Returns:
        List of dictionaries representing the entries
    """
    # Make sure we have entries
    if not self._entries:
        self.load()
    
    # Convert to dictionaries
    return [entry.model_dump() for entry in self._entries]

In [27]:
dataset_with_dataset_model.load_as_dicts()

[{'id': 0, 'name': 'test', 'description': 'test description'},
 {'id': 0, 'name': 'test', 'description': 'test description'},
 {'id': 0, 'name': 'test', 'description': 'test description'},
 {'id': 0, 'name': 'test', 'description': 'test description'},
 {'id': 0, 'name': 'test', 'description': 'test description'},
 {'id': 0, 'name': 'test', 'description': 'test description'},
 {'id': 0, 'name': 'test', 'description': 'test description'},
 {'id': 0, 'name': 'test', 'description': 'test description'},
 {'id': 0, 'name': 'test', 'description': 'test description'},
 {'id': 0, 'name': 'test', 'description': 'test description'}]

In [28]:
dataset_with_experiment_model.load_as_dicts()

[{'id': 0,
  'name': 'test',
  'description': 'test description',
  'tags': 'tag1',
  'result': '0.5'},
 {'id': 0,
  'name': 'test',
  'description': 'test description',
  'tags': 'tag1',
  'result': '0.5'},
 {'id': 0,
  'name': 'test',
  'description': 'test description',
  'tags': 'tag1',
  'result': '0.5'},
 {'id': 0,
  'name': 'test',
  'description': 'test description',
  'tags': 'tag1',
  'result': '0.5'},
 {'id': 0,
  'name': 'test',
  'description': 'test description',
  'tags': 'tag1',
  'result': '0.5'},
 {'id': 0,
  'name': 'test',
  'description': 'test description',
  'tags': 'tag1',
  'result': '0.5'},
 {'id': 0,
  'name': 'test',
  'description': 'test description',
  'tags': 'tag1',
  'result': '0.5'},
 {'id': 0,
  'name': 'test',
  'description': 'test description',
  'tags': 'tag1',
  'result': '0.5'},
 {'id': 0,
  'name': 'test',
  'description': 'test description',
  'tags': 'tag1',
  'result': '0.5'},
 {'id': 0,
  'name': 'test',
  'description': 'test description'

In [29]:
# | export
@patch
def to_pandas(self: Dataset) -> "pd.DataFrame":
    """Convert dataset to pandas DataFrame."""

    # Make sure we have data
    if not self._entries:
        self.load()
    
    # Convert entries to dictionaries
    data = [entry.model_dump() for entry in self._entries]
    return pd.DataFrame(data)

In [30]:
dataset_with_experiment_model.to_pandas()

,id,name,description,tags,result
0,0,test,test description,tag1,"(0, ., 5)"
1,0,test,test description,tag1,"(0, ., 5)"
2,0,test,test description,tag1,"(0, ., 5)"
3,0,test,test description,tag1,"(0, ., 5)"
4,0,test,test description,tag1,"(0, ., 5)"
5,0,test,test description,tag1,"(0, ., 5)"
6,0,test,test description,tag1,"(0, ., 5)"
7,0,test,test description,tag1,"(0, ., 5)"
8,0,test,test description,tag1,"(0, ., 5)"
9,0,test,test description,tag1,"(0, ., 5)"


In [31]:
# | export
@patch
def save(self: Dataset, item: BaseModelType) -> None:
    """Save changes to an item to the backend.
    
    Args:
        item: The item to save
    """
    if not isinstance(item, self.model):
        raise TypeError(f"Item must be an instance of {self.model.__name__}")
    
    # Check if the item has a row ID
    if not hasattr(item, "_row_id") or not item._row_id:
        # Try to find it in our entries by matching
        for i, entry in enumerate(self._entries):
            if id(entry) == id(item):  # Check if it's the same object
                if hasattr(entry, "_row_id") and entry._row_id:
                    item._row_id = entry._row_id
                    break
    
    if not hasattr(item, "_row_id") or not item._row_id:
        raise ValueError("Cannot save: item is not from this dataset or was not properly synced")
    
    # Update in backend
    self._backend.update_entry(item)
    
    # Update in local cache if needed
    self._update_local_entry(item)
    
@patch
def _update_local_entry(self: Dataset, item: BaseModelType) -> None:
    """Update an entry in the local cache.
    
    Args:
        item: The item to update
    """
    for i, entry in enumerate(self._entries):
        if hasattr(entry, "_row_id") and hasattr(item, "_row_id") and entry._row_id == item._row_id:
            # If it's not the same object, update our copy
            if id(entry) != id(item):
                self._entries[i] = item
            break

In [32]:
d = dataset_with_experiment_model[0]
d

ExperimentModel(id=0, name='test', description='test description', tags='tag1', result='0.5')

In [33]:
d.name = "updated name"
dataset_with_experiment_model.save(d)
dataset_with_experiment_model[0].name

'updated name'

In [35]:
dataset_with_experiment_model.load_as_dicts()

[{'id': 0,
  'name': 'updated name',
  'description': 'test description',
  'tags': 'tag1',
  'result': '0.5'},
 {'id': 0,
  'name': 'test',
  'description': 'test description',
  'tags': 'tag1',
  'result': '0.5'},
 {'id': 0,
  'name': 'test',
  'description': 'test description',
  'tags': 'tag1',
  'result': '0.5'},
 {'id': 0,
  'name': 'test',
  'description': 'test description',
  'tags': 'tag1',
  'result': '0.5'},
 {'id': 0,
  'name': 'test',
  'description': 'test description',
  'tags': 'tag1',
  'result': '0.5'},
 {'id': 0,
  'name': 'test',
  'description': 'test description',
  'tags': 'tag1',
  'result': '0.5'},
 {'id': 0,
  'name': 'test',
  'description': 'test description',
  'tags': 'tag1',
  'result': '0.5'},
 {'id': 0,
  'name': 'test',
  'description': 'test description',
  'tags': 'tag1',
  'result': '0.5'},
 {'id': 0,
  'name': 'test',
  'description': 'test description',
  'tags': 'tag1',
  'result': '0.5'},
 {'id': 0,
  'name': 'test',
  'description': 'test desc

In [36]:
# | export
@patch
def get(self: Dataset, field_value: t.Any, field_name: str = "_row_id") -> t.Optional[BaseModelType]:
    """Get an entry by field value.
    
    Args:
        field_value: The value to match
        field_name: The field to match against (default: "_row_id")
        
    Returns:
        The matching model instance or None if not found
    """
    # Check if we need to load entries
    if not self._entries:
        self.load()
    
    # Search in local entries first
    for entry in self._entries:
        if hasattr(entry, field_name) and getattr(entry, field_name) == field_value:
            return entry
    
    # If not found, try to get from backend
    if field_name == "_row_id":
        # Special case for row IDs
        for entry in self._entries:
            if hasattr(entry, "_row_id") and entry._row_id == field_value:
                return entry
    else:
        # Use backend to search
        return self._backend.get_entry_by_field(field_name, field_value, self.model)
    
    return None

In [37]:
d._row_id

'36eeed65-2105-4900-b9bb-bb42ddc35820'

In [38]:
dataset_instance = dataset_with_experiment_model.get(d._row_id)
dataset_instance

ExperimentModel(id=0, name='updated name', description='test description', tags='tag1', result='0.5')

In [39]:
# | export
@patch
def to_pandas(self: Dataset) -> "pd.DataFrame":
    """Convert dataset to pandas DataFrame.
    
    Returns:
        pd.DataFrame: A DataFrame containing all entries
    """
    # Make sure we have data
    if not self._entries:
        self.load()
    
    # Convert entries to dictionaries
    data = [entry.model_dump() for entry in self._entries]
    return pd.DataFrame(data)